In [3]:
# deploy.py
import os
import vertexai
from dotenv import load_dotenv
from scrap_translate.agent import root_agent

# Load environment variables from .env file for local execution of this script
load_dotenv()

# --- Configuration ---
# Replace with your real values, or set them in a .env file.
GCP_PROJECT_ID = "alpha-code-461805"
GCP_LOCATION = "us-central1"
# e.g., "gs://your-gcs-staging-bucket"
STAGING_BUCKET_URI ="gs://a2a_search_agent"
AGENT_DISPLAY_NAME = "Scrap and Translate Agent"
AGENT_DESCRIPTION = "An agent that can list and call other agents to perform tasks."

if not all([GCP_PROJECT_ID, GCP_LOCATION, STAGING_BUCKET_URI]):
    raise ValueError(
        "Please set GOOGLE_CLOUD_PROJECT, GOOGLE_CLOUD_LOCATION, and STAGING_BUCKET_URI "
        "in your environment or a .env file."
    )

In [4]:
import sys
!{sys.executable} -m pip install cloudpickle
import cloudpickle


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
# --- 1. Initialize Vertex AI SDK ---
vertexai.init(
    project=GCP_PROJECT_ID,
    location=GCP_LOCATION,
    staging_bucket=STAGING_BUCKET_URI
)

# --- 2. Define Deployment Configuration ---
# List of PyPI packages required by your agent.
# The Reasoning Engine service will install these dependencies.
requirements = [
    # google-cloud-aiplatform with reasoningengine and adk extras is needed
    "google-cloud-aiplatform[reasoningengine,adk]",
    "httpx",
    "python-dotenv",
    "a2a-client",  # Based on your agent's imports
]

# The directory containing your agent's Python package.
# This will be packaged and uploaded.
extra_packages = [
    "./scrap_translate",
]

In [6]:
#Use reasoning_engines.AdkApp() to wrap your agent to make it deployable to Agent Engine


from vertexai.preview import reasoning_engines

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

In [7]:
#Create Session Locally
session = app.create_session(user_id="u_123")
session

2025-07-17 18:19:25,327 - DEBUG - asyncio - Using selector: KqueueSelector
2025-07-17 18:19:25,406 - DEBUG - google.auth._default - Checking None for explicit credentials as part of auth process...
2025-07-17 18:19:25,407 - DEBUG - google.auth._default - Checking Cloud SDK credentials as part of auth process...


Session(id='6152df4d-1630-4172-b733-49b3e272f34c', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1752756566.564623)

In [8]:
for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message="Summarize the following BLOG URL : https://medium.com/google-cloud/the-agent-advantage-why-smart-ai-is-better-than-simple-bots-and-how-to-know-whats-real-62bebc28a0e6",
    ):
    print(event)

2025-07-17 18:19:28,424 - DEBUG - asyncio - Using selector: KqueueSelector
2025-07-17 18:19:28,429 - INFO - google_genai._api_client - The project/location from the environment variables will take precedence over the API key from the environment variables.
2025-07-17 18:19:28,524 - INFO - google_adk.google.adk.models.google_llm - Sending out request, model: gemini-2.5-pro, backend: vertex, stream: False
2025-07-17 18:19:28,526 - INFO - google_adk.google.adk.models.google_llm - 
LLM Request:
-----------------------------------------------------------
System Instruction:
You are a root orchestrator agent. You have two tools:
1) list_agents() → Use this tool to see a list of all available agents and their capabilities.
2) call_agent(agent_name: str, message: str) → Use this tool to send a message to a specific agent by its name and get its response.
Use these tools to fulfill user requests by discovering and interacting with other agents as needed.


You are an agent. Your internal name i

{'content': {'parts': [{'function_call': {'id': 'adk-84af9ca3-07a1-4512-8938-529639dc550f', 'args': {}, 'name': 'list_agents'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 3, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 3}], 'prompt_token_count': 260, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 260}], 'thoughts_token_count': 894, 'total_token_count': 1157, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-c033834c-720e-4193-a922-84a823252886', 'author': 'root_orchestrator', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'jHAePpsz', 'timestamp': 1752756568.429337}


2025-07-17 18:19:43,799 - DEBUG - httpcore.http11 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'content-type', b'application/json'), (b'x-cloud-trace-context', b'34f37298310511d15dba0e3962e80feb;o=1'), (b'date', b'Thu, 17 Jul 2025 12:49:43 GMT'), (b'server', b'Google Frontend'), (b'Content-Length', b'772'), (b'Alt-Svc', b'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000')])
2025-07-17 18:19:43,800 - INFO - httpx - HTTP Request: GET https://search-agent-100889782425.us-central1.run.app/.well-known/agent.json "HTTP/1.1 200 OK"
2025-07-17 18:19:43,800 - DEBUG - httpcore.http11 - receive_response_body.started request=<Request [b'GET']>
2025-07-17 18:19:43,801 - DEBUG - httpcore.http11 - receive_response_body.complete
2025-07-17 18:19:43,802 - DEBUG - httpcore.http11 - response_closed.started
2025-07-17 18:19:43,803 - DEBUG - httpcore.http11 - response_closed.complete
2025-07-17 18:19:43,803 - INFO - a2a.client.client - Successfully fetched agent card data fro

Fetched agent 'MultiURLBrowser' from registry at https://search-agent-100889782425.us-central1.run.app
Agent data: additionalInterfaces=None capabilities=AgentCapabilities(extensions=None, pushNotifications=None, stateTransitionHistory=None, streaming=True) defaultInputModes=['text'] defaultOutputModes=['text'] description='Agent designed to efficiently scrape specified content from multiple URLs or single URL provided by the user.' documentationUrl=None iconUrl=None name='MultiURLBrowser' preferredTransport=None protocolVersion='0.2.5' provider=None security=None securitySchemes=None skills=[AgentSkill(description='Agent to scrape content from the URLs specified by the user.', examples=['Scrape the URL: https://medium.com/@neeraj_agrawal/an-ai-travel-agent-in-action-a-detailed-look-at-how-two-agents-plan-a-trip-86a1735368e1', 'Extract data from: https://www.example.com/page1 and https://www.example.com/page2'], id='MultiURLBrowser', inputModes=None, name='MultiURLBrowser_Agent', outpu

2025-07-17 18:19:44,781 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): oauth2.googleapis.com:443
2025-07-17 18:19:44,963 - DEBUG - urllib3.connectionpool - https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
2025-07-17 18:19:44,967 - DEBUG - httpcore.connection - connect_tcp.started host='us-central1-aiplatform.googleapis.com' port=443 local_address=None timeout=None socket_options=None
2025-07-17 18:19:44,989 - DEBUG - httpcore.connection - connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x12ba31520>
2025-07-17 18:19:44,990 - DEBUG - httpcore.connection - start_tls.started ssl_context=<ssl.SSLContext object at 0x12bfc85d0> server_hostname='us-central1-aiplatform.googleapis.com' timeout=None
2025-07-17 18:19:45,027 - DEBUG - httpcore.connection - start_tls.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x12ba20770>
2025-07-17 18:19:45,027 - DEBUG - httpcore.http11 - send_request_headers.starte

...

Fetched agent 'MultiURLBrowser' from registry at https://search-agent-100889782425.us-central1.run.app
Agent data: additionalInterfaces=None capabilities=AgentCapabilities(extensions=None, pushNotifications=None, stateTransitionHistory=None, streaming=True) defaultInputModes=['text'] defaultOutputModes=['text'] description='Agent designed to efficiently scrape specified content from multiple URLs or single URL provided by the user.' documentationUrl=None iconUrl=None name='MultiURLBrowser' preferredTransport=None protocolVersion='0.2.5' provider=None security=None securitySchemes=None skills=[AgentSkill(description='Agent to scrape content from the URLs specified by the user.', examples=['Scrape the URL: https://medium.com/@neeraj_agrawal/an-ai-travel-agent-in-action-a-detailed-look-at-how-two-agents-plan-a-trip-86a1735368e1', 'Extract data from: https://www.example.com/page1 and https://www.example.com/page2'], id='MultiURLBrowser', inputModes=None, name='MultiURLBrowser_Agent', outpu

2025-07-17 18:20:14,686 - DEBUG - httpcore.http11 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'content-type', b'application/json'), (b'x-cloud-trace-context', b'0447b9c9488692997791afa9a2921cf2'), (b'date', b'Thu, 17 Jul 2025 12:50:14 GMT'), (b'server', b'Google Frontend'), (b'Content-Length', b'3405'), (b'Alt-Svc', b'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000')])
2025-07-17 18:20:14,686 - INFO - httpx - HTTP Request: POST https://search-agent-5urnfhd5eq-uc.a.run.app "HTTP/1.1 200 OK"
2025-07-17 18:20:14,687 - DEBUG - httpcore.http11 - receive_response_body.started request=<Request [b'POST']>
2025-07-17 18:20:14,687 - DEBUG - httpcore.http11 - receive_response_body.complete
2025-07-17 18:20:14,688 - DEBUG - httpcore.http11 - response_closed.started
2025-07-17 18:20:14,688 - DEBUG - httpcore.http11 - response_closed.complete
2025-07-17 18:20:14,690 - INFO - google_genai._api_client - The project/location from the environment variables will take prec

Response from send_message: root=SendMessageSuccessResponse(id=1, jsonrpc='2.0', result=Task(artifacts=None, contextId='74ea7c75-16ef-4603-b192-040668496bad', history=[Message(contextId='74ea7c75-16ef-4603-b192-040668496bad', extensions=None, kind='message', messageId='4aef194c950048bf91381b1ecde920f4', metadata=None, parts=[Part(root=TextPart(kind='text', metadata=None, text='Summarize the following BLOG URL : https://medium.com/google-cloud/the-agent-advantage-why-smart-ai-is-better-than-simple-bots-and-how-to-know-whats-real-62bebc28a0e6'))], referenceTaskIds=None, role=<Role.user: 'user'>, taskId='6d77641b-a173-41c5-86ec-de039f252da9'), Message(contextId='74ea7c75-16ef-4603-b192-040668496bad', extensions=None, kind='message', messageId='20f07ea4-0489-468e-bcd0-035a18605620', metadata=None, parts=[Part(root=TextPart(kind='text', metadata=None, text='Processing the web crawling request...'))], referenceTaskIds=None, role=<Role.agent: 'agent'>, taskId='6d77641b-a173-41c5-86ec-de039f25

2025-07-17 18:20:15,211 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): oauth2.googleapis.com:443
2025-07-17 18:20:15,399 - DEBUG - urllib3.connectionpool - https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
2025-07-17 18:20:15,400 - DEBUG - httpcore.connection - connect_tcp.started host='us-central1-aiplatform.googleapis.com' port=443 local_address=None timeout=None socket_options=None
2025-07-17 18:20:15,416 - DEBUG - httpcore.connection - connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x12bff0830>
2025-07-17 18:20:15,417 - DEBUG - httpcore.connection - start_tls.started ssl_context=<ssl.SSLContext object at 0x12bed4850> server_hostname='us-central1-aiplatform.googleapis.com' timeout=None
2025-07-17 18:20:15,452 - DEBUG - httpcore.connection - start_tls.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x12ba19d30>
2025-07-17 18:20:15,453 - DEBUG - httpcore.http11 - send_request_headers.starte

{'content': {'parts': [{'text': 'This article from the Google Cloud - Community blog explains the difference between AI agents and simpler bots, highlighting the "agent advantage." Here\'s a summary:\n\n**AI agents are a major advancement over traditional chatbots.** While bots follow pre-programmed scripts, AI agents are autonomous systems that can understand goals, create plans, and take independent action. They are described as "digital teammates" capable of handling complex and important tasks.\n\nAn AI agent has three key components:\n\n*   **An LLM "Brain":** A large language model (like Google\'s Gemini) that provides the core intelligence for understanding, reasoning, and planning.\n*   **Memory:** Both short-term memory for the current task and long-term memory to recall past interactions and learn over time.\n*   **Tool Use:** The ability to connect to and use other software, APIs, and data sources, allowing the agent to perform actions in the real world (e.g., accessing cust

In [9]:
from vertexai import agent_engines

remote_agent = agent_engines.create(
    app,
    display_name=AGENT_DISPLAY_NAME,
    description=AGENT_DESCRIPTION,
    requirements=["google-cloud-aiplatform[agent_engines,adk]", "a2a-sdk"],
    extra_packages=["scrap_translate"]
)

Identified the following requirements: {'google-cloud-aiplatform': '1.103.0', 'cloudpickle': '3.1.1', 'pydantic': '2.11.7'}


2025-07-17 18:20:33,411 - INFO - vertexai.agent_engines - Identified the following requirements: {'google-cloud-aiplatform': '1.103.0', 'cloudpickle': '3.1.1', 'pydantic': '2.11.7'}


The following requirements are missing: {'cloudpickle', 'pydantic'}


2025-07-17 18:20:33,452 - WARNING - vertexai.agent_engines - The following requirements are missing: {'cloudpickle', 'pydantic'}


The following requirements are appended: {'pydantic==2.11.7', 'cloudpickle==3.1.1'}


...

2025-07-17 18:24:13,578 - INFO - vertexai.agent_engines - AgentEngine created. Resource name: projects/100889782425/locations/us-central1/reasoningEngines/631796973506134016


To use this AgentEngine in another session:


2025-07-17 18:24:13,580 - INFO - vertexai.agent_engines - To use this AgentEngine in another session:


agent_engine = vertexai.agent_engines.get('projects/100889782425/locations/us-central1/reasoningEngines/631796973506134016')


2025-07-17 18:24:13,581 - INFO - vertexai.agent_engines - agent_engine = vertexai.agent_engines.get('projects/100889782425/locations/us-central1/reasoningEngines/631796973506134016')


In [10]:
for event in remote_agent.stream_query(
    user_id="user_123",
    message="Summarize the following BLOG URL : https://medium.com/google-cloud/the-agent-advantage-why-smart-ai-is-better-than-simple-bots-and-how-to-know-whats-real-62bebc28a0e6"
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-f2f81aba-0db0-4bfd-86c5-fee160725382', 'args': {}, 'name': 'list_agents'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 3, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 3}], 'prompt_token_count': 260, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 260}], 'thoughts_token_count': 133, 'total_token_count': 396, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-3242af82-8027-42ad-9cc7-dd74761249dc', 'author': 'root_orchestrator', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': '46318d9e-7fb6-473d-9149-ba6f18f4bb6d', 'timestamp': 1752756862.491556}
{'content': {'parts': [{'function_response': {'id': 'adk-f2f81aba-0db0-4bfd-86c5-fee160725382', 'name': 'list_agents', 'response': {'result': {'capabilities': {'streaming': True}, 'defaultInputModes': ['text'], 'defaultOutputModes': ['text'], 'description': 'Agent design